### w2v50をテキストにする

In [1]:
cd ../data/external/

/home/saho/project/wovle/data/external


In [3]:
ls

0         1gm-0000             dictation-kit-v4.4/   w2v200.pd
0.filted  1.mecab              filelist              w2v200.txt
0.mecab   CJLC-0.1/            general_words_300.pd  w2v50_fitToDic.tsv
1         csj.pdp.htkdic       gm_sorted.csv         w2v50.pd
1.filted  csj.pdp.htkdic.utf8  w2v200_fitToDic.tsv   w2v50.tsv


In [5]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('w2v50/word2vec.gensim.model')
len(model.wv.vocab)

335476

In [6]:
with open('w2v50.tsv','w') as fh:
    for (k,v) in monitor(model.wv.vocab.items()):
        fh.write(k+"\t"+"\t".join([str(e) for e in model.wv[k]])+"\n")

100%|██████████| 335476/335476 [00:38<00:00, 8754.04it/s]


In [9]:
rm -r w2v50/

In [10]:
ls

1gm-0000   csj.pdp.htkdic       filelist    w2v200.txt
CJLC-0.1/  dictation-kit-v4.4/  index.html  w2v50.tsv


### 辞書（csj.pdp.htkdic）に無いものは削除

In [12]:
!nkf -w ./csj.pdp.htkdic > ./csj.pdp.htkdic.utf8

In [13]:
!head ./csj.pdp.htkdic.utf8

In [4]:
!head ../raw/hybrid.htkdic

<s>	[]	sp_S
</s>	[。]	sp_S
、+補助記号	[、]	sp_S
×+名詞	[×]	b_B a_I ts_I u_E
Δ+記号	[Δ]	d_B e_I r_I u_I t_I a_E
Θ+記号	[Θ]	sh_B i:_I t_I a_E
Σ+記号	[Σ]	sh_B i_I g_I u_I m_I a_E
Φ+記号	[Φ]	f_B a_I i_E
Ψ+記号	[Ψ]	p_B u_I s_I a_I i_E
Ω+名詞	[Ω]	o:_B m_I u_E


In [5]:
!wc -l ./csj.pdp.htkdic.utf8

38598 ./csj.pdp.htkdic.utf8


In [6]:
!wc -l ../raw/hybrid.htkdic

190535 ../raw/hybrid.htkdic


In [7]:
words = set()

with open('../raw/hybrid.htkdic') as hh:
    for line in monitor(hh):
        words.add(line.split('\t')[0].split('+')[0])

with open('w2v50.tsv') as v50h:
    with open('w2v200.txt') as v200h:
        with open('w2v50_fitToDic.tsv','w') as w50h:
            with open('w2v200_fitToDic.tsv','w') as w200h:
                for line in monitor(v50h):
                    w = line.split('\t')[0]
                    if w in words:
                        w50h.write(w+'\t'+'\t'.join(line.split('\t')[1:]))
                for line in monitor(v200h):
                    w = line.split(' ')[0].replace('[','').replace(']','') # 括弧は無いものとして比較
                    if w in words:
                        w200h.write(w+'\t'+'\t'.join(line.split(' ')[1:]))

190535it [00:00, 390069.02it/s]
335476it [00:05, 60867.71it/s]
1015475it [00:45, 22547.21it/s]


In [8]:
!wc -l ./w2v50_fitToDic.tsv

124841 ./w2v50_fitToDic.tsv


In [9]:
!wc -l ./w2v200_fitToDic.tsv

228113 ./w2v200_fitToDic.tsv


### w2v200の括弧重複の平均を取る

#### dataframeとして読み込み

In [2]:
w2v50 = pd.read_csv("w2v50_fitToDic.tsv",sep="\t",header=None)

In [3]:
w2v200 = pd.read_csv("w2v200_fitToDic.tsv",sep="\t",header=None)

In [4]:
w2v = [w2v50, w2v200]

In [5]:
for w in w2v: display(w.shape)

(124841, 51)

(228113, 201)

In [6]:
w2v[0].to_pickle('w2v50.pd')

In [7]:
w2v[1].to_pickle('w2v200.pd')

#### 重複除外

In [8]:
w = w2v[1]

In [9]:
def disp(*v):
    for e in v: display(e)

In [10]:
disp(
    w.ix[[0],:],
    w.shape)

/home/saho/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,の,-0.991739,0.433873,-1.634327,-2.637119,1.87341,-0.68232,1.400241,2.555166,-2.150506,...,-1.206327,0.912507,-0.449425,-0.685022,-0.067702,-0.439921,-1.823313,-0.582093,-0.289067,-0.953114


(228113, 201)

In [11]:
w.ix[w[0] == "トヨタ"]

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
8571,トヨタ,0.053538,-0.458620,2.269142,2.213828,1.045843,-0.385347,1.634038,2.483560,-1.511059,...,-0.565592,-1.320724,1.17779,-0.255538,1.393715,-0.182194,0.562751,-0.150641,0.373629,0.638203
28411,トヨタ,-0.306716,-1.427696,1.110670,1.346888,-0.350039,-0.451238,1.047367,1.017068,-1.072866,...,-0.431226,-0.514442,1.05925,-0.159481,0.126721,-0.407040,-0.150029,0.352745,0.327678,-0.481530


In [12]:
w = w.groupby([0]).mean()

In [13]:
w.head()

,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0,,,,,,,,,,,,,,,,,,,,,
%,-0.549040,1.596468,0.238416,0.001363,-0.142438,-0.191435,0.030494,0.528204,-0.213250,-0.204060,...,-0.795360,0.344991,-0.386084,1.248316,-0.788896,-0.637377,-2.325006,-0.462570,-0.094412,0.217285
((,-1.545232,0.765162,1.382379,0.638051,-0.518116,-1.432012,-0.136455,1.345916,-0.561603,0.218456,...,1.970857,-1.016529,0.168405,-1.308969,-0.879456,-1.285879,-0.701499,-1.625317,0.777812,-0.514972
(),0.246733,0.503330,0.664697,-0.229053,0.120119,-1.470995,-0.176881,2.603051,-1.151863,0.017401,...,0.225183,-1.554308,-0.222451,0.913195,-0.311706,1.817891,-1.502593,1.459932,-1.276487,-1.313961
(;),-0.112726,0.068179,0.090609,-0.003347,0.004794,0.007828,-0.137118,0.011520,-0.025337,0.040023,...,0.046870,-0.064753,0.051109,-0.021175,-0.045148,0.035360,-0.043468,-0.088414,-0.102241,-0.026822
(^_^),-0.021000,0.045577,0.050728,0.001926,-0.012316,0.022077,-0.051286,0.001320,-0.036536,-0.040432,...,0.018945,-0.050968,-0.073365,0.054868,-0.046374,0.022102,-0.011911,-0.073225,-0.061821,0.005072


In [14]:
w.shape

(177821, 200)

In [15]:
w.to_pickle('./w2v200.pd')

In [16]:
w = w2v[0]

In [17]:
w.head()

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,GO,0.017380,-0.320580,-0.233787,-0.241856,0.140260,0.061071,0.097929,-0.237189,0.209079,...,0.206805,0.097653,0.084598,0.055724,-0.159079,0.113399,-0.005980,0.135504,0.073637,-0.140329
1,ガリバー,0.091114,-0.294387,0.021036,-0.036468,0.101153,0.058209,0.147456,0.059062,0.146042,...,0.239939,-0.006300,0.099647,0.036027,-0.008679,0.039122,-0.049217,0.203507,0.180088,-0.169165
2,くん,0.060573,-0.231098,0.012265,-0.118655,-0.045415,-0.082472,0.005965,-0.031879,0.066911,...,0.133983,-0.022876,0.006493,0.064899,-0.067395,-0.020905,-0.154779,0.175292,0.191491,-0.281831
3,ゴーゴー,0.076387,-0.237495,-0.021150,-0.105880,0.030181,0.174667,0.054576,-0.078703,0.109202,...,0.157529,0.116068,0.040623,0.034536,-0.133727,0.097389,-0.087429,0.222188,0.100252,-0.260785
4,は,0.206208,0.238194,0.164277,-0.219658,0.163085,-0.229095,0.174790,0.049674,-0.146848,...,-0.105047,-0.001622,0.188861,-0.127376,0.117224,0.107232,0.254831,0.029983,0.024894,0.234945


In [18]:
w = w.groupby([0]).mean()

In [19]:
w.shape

(124837, 50)

In [20]:
w.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,,,,,,,,,,,,,,,,,,,,,
%,-0.113895,-0.091425,-0.370241,-0.086551,0.207204,0.136094,0.103111,0.117269,-0.056832,-0.152773,...,-0.190907,0.152524,-0.272115,-0.087875,0.087905,0.130348,-0.108683,-0.025001,-0.203008,-0.159233
((,-0.279528,-0.051735,-0.128621,0.016157,-0.131595,-0.137277,0.216305,-0.055012,0.050739,0.043952,...,-0.124552,0.348924,0.113755,0.039081,0.031400,-0.103040,-0.213814,0.135074,-0.146652,0.173620
(),-0.073833,0.194739,-0.028522,0.025788,-0.016889,0.126192,0.050237,-0.070981,0.108441,0.022274,...,0.243351,-0.067128,0.316693,-0.083357,0.158637,0.048287,-0.012257,-0.093431,-0.043053,0.097612
007,-0.087771,-0.031139,-0.199600,-0.021349,-0.049920,0.012160,0.294963,-0.028720,0.103570,0.120526,...,0.228207,-0.071338,0.122428,-0.068095,0.005586,-0.126803,0.015559,0.086991,-0.066008,-0.036388
><,-0.009978,-0.208440,0.211985,0.058901,-0.271243,-0.244008,-0.151502,0.090421,-0.007092,0.120083,...,-0.083811,0.139420,-0.095053,-0.021689,0.060579,-0.005183,-0.099164,0.088626,0.034882,-0.107283


In [21]:
w.to_pickle('./w2v50.pd')

In [22]:
w2v = []


In [23]:
w2v.append(pd.read_pickle('w2v50.pd'))
w2v.append(pd.read_pickle('w2v200.pd'))

### 非自立語の削除

In [24]:
for (i,w) in enumerate(w2v):
    with open(str(i),"w") as fh:
        for e in w.index:
            fh.write(e+"\n")

In [25]:
!mecab 0 > 0.mecab
!mecab 1 > 1.mecab

In [26]:
!head 0.mecab

%	名詞,接尾,助数詞,*,*,*,%,パーセント,パーセント
EOS
((	名詞,固有名詞,一般,*,*,*,((,カッコカッコ,カッコカッコ
EOS
()	名詞,固有名詞,一般,*,*,*,(),カッコ,カッコ
EOS
007	名詞,固有名詞,人名,一般,*,*,007,ゼロゼロセブン,ゼロゼロセブン
EOS
><	名詞,固有名詞,一般,*,*,*,><,カオモジ,カオモジ
EOS


In [27]:
!head 1.mecab

%	名詞,接尾,助数詞,*,*,*,%,パーセント,パーセント
EOS
((	名詞,固有名詞,一般,*,*,*,((,カッコカッコ,カッコカッコ
EOS
()	名詞,固有名詞,一般,*,*,*,(),カッコ,カッコ
EOS
(;)	記号,一般,*,*,*,*,(・・;),カオモジ,カオモジ
EOS
(^_^)	記号,一般,*,*,*,*,(^_^),エガオ,エガオ
EOS


In [28]:
 !cat 0.mecab |\
    grep -v "EOS" |\
    grep -v "助詞" |\
    grep -v "助動詞" |\
    grep -v "記号" |\
    grep -v "フィラー" |\
    grep -v "感動詞" |\
    grep -v "接頭詞" |\
    grep -v "接続詞" |\
    grep -v "連体詞" |\
    grep -v "副詞" |\
    grep -v "代名詞" |\
    grep -v "非自立" |\
    grep -v "この" \
    > 0.filted

In [29]:
 !cat 1.mecab |\
    grep -v "EOS" |\
    grep -v "助詞" |\
    grep -v "助動詞" |\
    grep -v "記号" |\
    grep -v "フィラー" |\
    grep -v "感動詞" |\
    grep -v "接頭詞" |\
    grep -v "接続詞" |\
    grep -v "連体詞" |\
    grep -v "副詞" |\
    grep -v "代名詞" |\
    grep -v "非自立" |\
    grep -v "この" \
    > 1.filted

In [30]:
!wc -l 0.filted

122514 0.filted


In [31]:
!wc -l 1.filted

176161 1.filted


In [32]:
w2v_word = [[],[]]
with open('0.filted') as fh:
    w2v_word[0] = [e.split('\t')[0] for e in fh]
with open('1.filted') as fh:
    w2v_word[1] = [e.split('\t')[0] for e in fh]
        

In [33]:
for w in w2v: display(w.shape)

(124837, 50)

(177821, 200)

In [34]:
w2v[0] = w2v[0][w2v[0].index.isin(w2v_word[0])]

In [35]:
w2v[1] = w2v[1][w2v[1].index.isin(w2v_word[1])]

In [36]:
for w in w2v: display(w.shape)

(120804, 50)

(170374, 200)

In [37]:
w2v[0].to_pickle('./w2v50.pd')
w2v[1].to_pickle('./w2v200.pd')